## Scraping Domains
This task was given after an interview to evaluate technical skills

The main objective is to scrape a list of 10k domains and match them with a database of companies info.

In [2]:
import re
import os
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

Read Domains csv to a pandas DF

In [4]:
domains_df = pd.read_csv('domains.csv', header=0, delimiter=',')
domains_df.head()

,domain
0,issos.nl
1,beeldenbouwers.nl
2,janbuisbaarn.nl
3,wilkbeglazing.nl
4,sjerpstuinen.nl


Pandas Df to a list of lists, each element is a row of the DF

In [5]:
domains_list = domains_df.values.tolist()
domains_list[:2]

[['issos.nl'], ['beeldenbouwers.nl']]

Read Companies csv to a DF

In [18]:
companies_df = pd.read_csv('companies.csv', header=0, delimiter=',')
companies_df.head()

,address1,city,postal_code,country,founding_date,name,company_number,phone_number,unique_id
0,Synagogeplantsoen 64,Meerssen,6231KK,NL,2008-01-01,Eigen Kracht,14098137,+31433640535,600121816
1,Nieuwkerk 2,Goirle,5051PN,NL,2013-01-01,Klooster Nieuwkerk Goirle B.V.,57677174,+31135302510,602107256
2,'t Hambroek 16,Borculo,7271HA,NL,2001-01-01,Adfistro,08097700,+31545274166,517367866
3,Venrayseweg 4,Wanssum,5861BD,NL,2008-01-01,Skymeubel V.O.F.,14104757,NaN,600288132
4,Van Eedenstraat 9,Haarlem,2012EL,NL,2021-01-01,Oass B.V.,81827660,+31235311077,604825839


After setting up the scrapy tutorial and testing the library along with some domains examples it is clear that not all domains have the full company data in their main page. Additionaly the format of the domains HTML page is often different and cues using the html code, like used in the scrapy tutorial, are impossible to use when extracting for all domains.

For example:  
"footer_content = response.css('div.w-layout-grid.main-grid.footer *::text').getall()" - this only works if the footer has this specific div hierarchy found for the https://issos.nl domain


Unique info that can be used to match domains with companies seems to be 
- **phone number**
- **company number**
- **name**
- **address** ?

Companies could share city, postal code, country, founding date, so unusable for unique identifiers, maybe address can be used?

From very small tests, its very common that the main page includes phone number, some contain the company number as well, and the name of the company can also be found usually in the main page, but hard to extract it as no regular expression can be made for it.

### PCA analysis on the companies database

In [21]:
def nan_ratio(df, df_name):
    """Calculate the ratio of NAN rows in a datasets columns
    Args: 
        df - dataframe
        df_name - string, name of the dataset for the print
    """
    
    print(f'Missing data ratio in {df_name}:\n')
    df_nan_ratio = df.isna().sum() / len(df) * 100
    df_nan_ratio.sort_values(ascending=False,inplace=True)
    df_nan_ratio = df_nan_ratio.to_frame(name='Nan Ratio %').T
    display(df_nan_ratio)

In [68]:
nan_ratio(companies_df, 'Companies')

Missing data ratio in Companies:



,phone_number,company_number,founding_date,city,postal_code,address1,country,name,unique_id
Nan Ratio %,36.16,2.4,0.29,0.02,0.01,0.0,0.0,0.0,0.0


**Phone number** is missing in 36% of the data, this is terrible as its the easiest data to extract with scrapy. **Company number** also has 2.4% missing data, from unique ids it seems only **address** and **name** are complete

In [69]:
def nan_ratio_multi_cols(df, df_name, cols):
    """Calculate the ratio of NAN rows in both columns a dataset
    Args: 
        df - dataframe
        df_name - string, name of the dataset for the print
        cols - list of cols to consider null together
    """

    nan_check = df[cols].isna()
    both_nan = nan_check.all(axis=1)
    df_nan_ratio = both_nan.sum() / len(df) * 100

    print(f'Ratio of rows where "{'" and "'.join(cols)}" are Nan: {float(df_nan_ratio)} for dataset "{df_name}"')  

In [70]:
cols = ['phone_number', 'company_number']
nan_ratio_multi_cols(companies_df, 'Companies', cols)

Ratio of rows where "phone_number" and "company_number" are Nan: 0.12 for dataset "Companies"


This means phone number and company number columns alone cant be used to match all companies, address must be used.

Checking for uniqueness of identifier columns

In [124]:
def unique_ratio(df, df_name):
    """Calculate the ratio of unique rows in a datasets columns, excluding Nan
    Args: 
        df - dataframe
        df_name - string, name of the dataset for the print
    """
    
    print(f'Unique data ratio in {df_name}:\n')
    companies_df.dropna(inplace=True)
    dict_unique_ratio = {col: companies_df[col].nunique() / len(companies_df) * 100 for col in companies_df.columns}
    df_unique_ratio = pd.DataFrame(dict_unique_ratio, index=['Unique Ratio']).T
    df_unique_ratio.sort_values(by='Unique Ratio', ascending=False, inplace=True)
    display(df_unique_ratio.T)

In [125]:
unique_ratio(companies_df, 'Companies')

Unique data ratio in Companies:



,unique_id,company_number,name,phone_number,address1,postal_code,city,founding_date,country
Unique Ratio,100.0,100.0,99.934938,99.756018,98.503578,93.038386,19.795055,1.951854,0.162655


Only **Company number** is 100% unique in the data set, seems that **name**, **phone number** and **address** have some duplicates, but a very small percentage less than 1

In [126]:
duplicates = companies_df[companies_df.duplicated(keep=False)]
duplicates

,address1,city,postal_code,country,founding_date,name,company_number,phone_number,unique_id


There are no rows completely duplicated

In [132]:
duplicate_names = companies_df[companies_df.duplicated(subset=['name'], keep=False)].sort_values(by='name')
duplicate_names

,address1,city,postal_code,country,founding_date,name,company_number,phone_number,unique_id
937,Watervalweg 143,Ermelo,3851VB,NL,2001-01-01,Baron B.V.,08100543,+31341361552,517775499
4185,Kromstraat 80,Oss,5345AD,NL,1999-01-01,Baron B.V.,17117541,+31412627426,515120218
540,Koxkampseweg 10,Zaltbommel,5301KK,NL,2020-01-01,Duran B.V.,11026447,+31418515583,528854246
4715,Gessel 56,De Meern,3454MZ,NL,2013-01-01,Duran B.V.,57822239,+31306304444,602124926
11,Joop van Weezelhof 7,Amsterdam,1063MK,NL,1986-01-01,Holland & Barrett B.V.,16829476,+31206144866,516149604
8643,Roelantdreef 241,Utrecht,3562KH,NL,1986-01-01,Holland & Barrett B.V.,16829239,+31302628954,529818513
2422,Grote Haven 2 D,Haastrecht,2851BM,NL,2015-01-01,Regionale Dienst Openbare Gezondheidszorg Holl...,31370454,+31182501349,602676376
6487,Oranjeplaats 1 Unit A,Schoonhoven,2871TK,NL,2012-01-01,Regionale Dienst Openbare Gezondheidszorg Holl...,25332147,+31883083125,601685563


These companies have the same name, but different info for all else, should the domain be matched with both rows or one?  
This can be interpreted as the same companie with two offices in diff addresses and phone numbers

In [135]:
duplicate_phone = companies_df[companies_df.duplicated(subset=['phone_number'], keep=False)].sort_values(by='phone_number')
duplicate_phone

,address1,city,postal_code,country,founding_date,name,company_number,phone_number,unique_id
2475,Lange Bunder 5,Bavel,4854MB,NL,2022-01-01,Thuiszorg Groot Brabant B.V.,24249613,+31161437554,529758461
434,Lange Bunder 7,Bavel,4854MB,NL,2018-01-01,Villa Groot Brabant B.V.,71104151,+31161437554,603707264
6758,Verlengde Luijtenstraat 4,Lekkerkerk,2941CH,NL,2008-01-01,Luijten Verwarming B.V.,24444224,+31180664141,600327987
5197,Verlengde Luijtenstraat 4,Lekkerkerk,2941CH,NL,2007-01-01,P. Luijten Beheer B.V.,24427014,+31180664141,600116484
6998,Herikerbergweg 238,Amsterdam,1101CM,NL,2016-01-01,Sim Lease 5 B.V.,27372291,+31204064444,600861852
8081,Ankersmidplein 2,Zaandam,1506CK,NL,2008-01-01,Primark Netherlands B.V.,23744650,+31204064444,601467868
1417,Ettensestraat 19,Terborg,7061AA,NL,2007-01-01,Potomac Classics B.V.,09033767,+31315326419,514901631
267,Rijksweg 112,Gaanderen,7011EC,NL,2002-01-01,Potomac Motors Nederland B.V.,10040368,+31315326419,528894012
137,Hoogstraat 24 B 3,Nijkerk Gld,3862AL,NL,2011-01-01,Meacasa B.V.,51917416,+31333033500,601201162
4774,Hoogstraat 24 B 03,Nijkerk Gld,3862AL,NL,2017-01-01,Sock my Feet B.V.,72600446,+31333033500,603883921


This shows cases where companies have the same phone number, address and even name. Others have different name but share address, or other combinations

The only thing trully unique is the company number, but not all domains will show the company number, how to match them correctly?

To do:  
- Extract phone number using a regular expression from all the text in the domain
- Try to extract other info like Chamber of commerce company number and name - use copyright section at the bottom to get name?
- Combine and match the info extracted to generate a confidence percentage in the matches obtained, if all 3 match, 100%, if only one matches 75%, something like this

### Analysing Outputs

In [5]:
# scraped_out_df = pd.read_csv('scraping_test/scraping_test/spiders/output.csv')
# scraped_out_df.head()
# df_out = scraped_out_df[scraped_out_df['status'] == '200']
# df_out
# df_out.count()

,url,status,email,phone,address,copyright
0,zutphenaanzet.nl,Error: [Failure instance: Traceback: <class 't...,NaN,NaN,NaN,NaN
1,janbuisbaarn.nl,Error: [Failure instance: Traceback: <class 's...,NaN,NaN,NaN,NaN
2,adviesbedrijfcnossen.nl,Error: [Failure instance: Traceback: <class 's...,NaN,NaN,NaN,NaN
3,https://beeldenbouwers.nl,200,NaN,NaN,NaN,NaN
4,accountancydirect.nl,Error: [Failure instance: Traceback: <class 't...,NaN,NaN,NaN,NaN


Reached a limit when running for all 10k domains, possibly from too many open connections. (Tried TIMEOUT setting but causes other errors.) Batching into 1k files and running separately

In [6]:
file_path = 'domains.csv'  # Example path to the 10k rows CSV file
csv_data = pd.read_csv(file_path)

# Function to split the CSV into batches of 1000 rows each
def batch_csv(data, batch_size=1000):
    total_batches = len(data) // batch_size + (1 if len(data) % batch_size != 0 else 0)
    batches = []
    for i in range(total_batches):
        batch = data[i*batch_size:(i+1)*batch_size] # 0 to 1000, 1000 to 2000
        batch_file = f'./output/output_batch_{i+1}.csv'
        # batch.to_csv(batch_file, index=False)
        batches.append(batch_file)
    return batches

# Split the CSV into 1k row batches
batch_files = batch_csv(csv_data)

Now I have 10 domain files, can run each separately and output to the same file.  
Will write a bash script to loop the files and run the scrapy command outputing to different files

In [19]:
# scraped_out_df = pd.read_csv('scraping_test/scraping_test/spiders/output.csv')
# scraped_out_df.head()
# domains_OK = scraped_out_df[scraped_out_df['status'] == '200'].reset_index(drop=True)
# len(domains_OK) / len(scraped_out_df) * 100

,url,status,title,email,phone,address,copyright
0,zutphenaanzet.nl,Error: [Failure instance: Traceback: <class 't...,NaN,NaN,NaN,NaN,NaN
1,https://beeldenbouwers.nl,200,Domain Default page,NaN,NaN,NaN,NaN
2,https://gjgmedia.nl,200,Saus Media,NaN,NaN,NaN,NaN
3,janbuisbaarn.nl,Error: [Failure instance: Traceback: <class 's...,NaN,NaN,NaN,NaN,NaN
4,accountancydirect.nl,Error: [Failure instance: Traceback: <class 't...,NaN,NaN,NaN,NaN,NaN


70% of the domains have OK status, the other 30 % did not connect properly  
Errors found:
- DNS lookup failed
- Request failed for domain: non 200 responses
- TCP connection timed out

In [45]:
# domains_OK.count()
# domains_OK.sort_values(by='address').head(10)

index        2094
url          2094
status       2094
title        2094
email         942
phone         697
address        51
copyright     988
dtype: int64

The best match seems to come from the title and company name  
Trying Fuzzy matching after normalizing these

In [121]:
def normalize_title(title):
    title_l = title.lower()
    title_norm = re.sub(r'[^\w\s]', '', title_l)

    # Optional: Remove common stopwords (you can expand this list)
    stopwords = ['voor', 'en', 'de', 'het', 'een', 'van', 'te', 'in', 'op', 'bij', 'met', 
                'uit', 'om', 'ik', 'je', 'jij', 'hij', 'zij', 'ze', 'wij', 'we', 'mij', 
                'me', 'hem', 'haar', 'ons', 'hun', 'over', 'onder', 'naast', 'door', 
                'tegen', 'is', 'was', 'heb', 'heeft', 'hebben', 'worden', 'doen', 'kan', 
                'kunnen', 'maar', 'als', 'tot', 'dan', 'nu', 'weer', 'bv']
    
    text = ' '.join([word for word in title_norm.split() if word not in stopwords])
    
    return text


Testing

In [ ]:
# title_test = domains_OK['title'].iloc[264]
# title_test

# title_test_norm = normalize_title(title_test)
# title_test_norm

# # Load Companies names to test
# companies_df = pd.read_csv('companies.csv', header=0, delimiter=',')
# companies_df.head()
# # 5421
# company_test = companies_df['name'].iloc[5419]
# company_test
# company_test_norm = normalize_title(company_test)
# company_test_norm
# company_test2 = companies_df['name'].iloc[5422]
# company_test2
# company_test_norm2 = normalize_title(company_test2)
# company_test_norm2
# company_titles = [company_test, company_test2]
# company_titles_norm = [company_test_norm, company_test_norm2]
# match = process.extractOne(title_test_norm, company_titles_norm,  scorer=fuzz.token_sort_ratio)
# match
# best_match_title = company_titles[company_titles_norm.index(match[0])]
# best_match_title

## 

Seems to be working - generalizing for df and matching the domain to the companies unique ID

Read all batches into one df

In [3]:
num_batches = len(os.listdir('./output'))
df_scraped = pd.DataFrame()
for i in range(num_batches):
    df_scraped_batch = pd.read_csv(f'./output/output_batch_{i+1}.csv')
    # print(df_scraped_batch['url'].count())
    df_scraped = pd.concat([df_scraped, df_scraped_batch], ignore_index=True)

998
998
998
999
998
1000
1000
1000
1000
999


In [27]:
df_scraped.count()

url          9990
status       9990
title        7005
email        3193
phone        2402
address       138
copyright    3286
dtype: int64

In [166]:
df_scraped_OK = df_scraped[df_scraped['status'] == '200'].reset_index(drop=True)
df_scraped_OK.count()

url          7005
status       7005
title        7005
email        3193
phone        2402
address       138
copyright    3286
dtype: int64

In [29]:
df_companies = pd.read_csv('companies.csv')
df_companies.count()

address1          10000
city               9998
postal_code        9999
country           10000
founding_date      9971
name              10000
company_number     9760
phone_number       6384
unique_id         10000
dtype: int64

In [167]:
# Normalizing title and company name strings
def normalize(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    stopwords = ['voor', 'en', 'de', 'het', 'een', 'van', 'te', 'in', 'op', 'bij', 'met', 'uit', 'om', 'ik', 'je', 'jij', 'hij', 'zij', 'ze', 'wij', 'we', 'mij', 
                'me', 'hem', 'haar', 'ons', 'hun', 'over', 'onder', 'naast', 'door', 'tegen', 'is', 'was', 'heb', 'heeft', 'hebben', 'worden', 'doen', 'kan', 
                'kunnen', 'maar', 'als', 'tot', 'dan', 'nu', 'weer', 'bv']
    text = ' '.join([word for word in text.split() if word not in stopwords])
    return text

# Apply preprocessing to both scraped titles and company titles
df_scraped_OK['norm_title'] = df_scraped_OK['title'].apply(normalize)
df_companies['norm_name'] = df_companies['name'].apply(normalize)

In [168]:
df_scraped_OK[df_scraped_OK['norm_title'] == '']

,url,status,title,email,phone,address,copyright,norm_title
371,https://vdmaas.nl,200,-,NaN,NaN,NaN,NaN,
657,https://www.htdbranding.nl/,200,-,mailto:info@htdbranding.nl,NaN,NaN,© 2022 Scheelings&DochterMedia for HTDbranding,
1657,https://makeyourweb.nl,200,Over ons,NaN,NaN,NaN,NaN,
2643,https://www.studentcommunity.nl/,200,–,NaN,NaN,NaN,© 2020 Advando Project Control NL,
5002,https://smartinterim.nl,200,\n Over ons\n,NaN,NaN,NaN,NaN,
5736,https://www.flexinterim.nl/,200,|,mailto:info@flexinterim.nl,NaN,NaN,NaN,


In [36]:
# Fuzzy match with index tracking
def fuzzy_match_with_index(scraped_df, companies_df, threshold=0):
    matches = []
    for idx, scraped_title in enumerate(scraped_df['norm_title']):
        match = process.extractOne(scraped_title, companies_df['norm_name'], score_cutoff=threshold, scorer=fuzz.token_sort_ratio)
        if match:
            # Get the index of the matched company
            matched_idx = companies_df[companies_df['norm_name'] == match[0]].index[0]
            matches.append((idx, matched_idx, match[1]))  # (scraped title index, matched company index, score)
    return matches

# Perform the matching
matched_indexes = fuzzy_match_with_index(df_scraped_OK, df_companies, threshold=10)


In [37]:
matched_indexes[0:10]

[(0, np.int64(3367), 65),
 (1, np.int64(4797), 73),
 (2, np.int64(7843), 59),
 (3, np.int64(806), 57),
 (4, np.int64(2942), 51),
 (5, np.int64(33), 70),
 (6, np.int64(7857), 90),
 (7, np.int64(5199), 100),
 (8, np.int64(1215), 66),
 (9, np.int64(5981), 80)]

In [38]:
# Create a DataFrame of matches with indices
matched_df = pd.DataFrame(matched_indexes, columns=['scraped_index', 'company_index', 'score'])
matched_df.head()

,scraped_index,company_index,score
0,0,3367,65
1,1,4797,73
2,2,7843,59
3,3,806,57
4,4,2942,51


Analysing the Errors in Matching

In [65]:
matched_df[matched_df['score'] < 40].count()

scraped_index    22
company_index    22
score            22
dtype: int64

In [44]:
df_companies[['name', 'norm_name']].iloc[6351]

name         Vereniging van Eigenaars Tuinkade I (Tuinkade ...
norm_name    vereniging eigenaars tuinkade i tuinkade 2 32 ...
Name: 6351, dtype: object


In [46]:
df_scraped_OK[['title', 'norm_title']].iloc[855]

title         טמבור - הצבע של ישראל | מערכות צבע מתקדמות | ח...
norm_title    טמבור הצבע של ישראל מערכות צבע מתקדמות חומרי ג...
Name: 1197, dtype: object

In [47]:
df_scraped_OK[['title', 'norm_title']].iloc[4036]

title         Аренда сервера. Виртуальные и выделенные серверы
norm_title     аренда сервера виртуальные и выделенные серверы
Name: 5778, dtype: object

In [64]:
pd.set_option('display.max_colwidth', None)
print(df_scraped_OK[['title', 'norm_title']].iloc[245])
print(df_companies[['name', 'norm_name']].iloc[2381])

title         Dejavu Cars ''Because standard isn't enough'' WWW.DEJAVUCARS.EU - Dejavu Cars ''Because standard isn't enough'' WWW.DEJAVUCARS.EU
norm_title                    dejavu cars because standard isnt enough wwwdejavucarseu dejavu cars because standard isnt enough wwwdejavucarseu
Name: 333, dtype: object
name         Vereniging van Eigenaars Winkelcentrum Ridderhof I aan de Ridderhof en Jan Steenstraat te
norm_name                 vereniging eigenaars winkelcentrum ridderhof i aan ridderhof jan steenstraat
Name: 2381, dtype: object


In [66]:
len(matched_df[matched_df['score'] < 50]) / len(matched_df) * 100

13.447537473233403

There seem to be issues with geting different languages. Some sites don't have english option available, matching between languages gives out bad results.  
Possible modification is adding a translation method to first translate all date to the same language, then match.
Even then, bad matches seem to be a low percentage of overall data, 22 have less than 40 score. 50 score is already giving correct results

Dropping bad matches and keeping only unique matches with best score

In [68]:
df_matched_maxScore = matched_df.loc[matched_df.groupby('company_index')['score'].idxmax()]

In [78]:
matched_df['company_index'].nunique()

3875

In [91]:
matched_df.groupby('company_index').size().reset_index(name='count').sort_values(by='count', ascending=False)

,company_index,count
2378,5981,349
2249,5625,98
1198,2942,78
3816,9857,72
1635,4076,63
...,...,...
3845,9939,1
3846,9941,1
3849,9950,1
3851,9954,1


In [86]:
print(df_companies[['name', 'norm_name']].iloc[5981])

name         Danone
norm_name    danone
Name: 5981, dtype: object


It seems some company names are matching with too many scraped titles, this is an undesired behaviour, could try to remove values from the company names pool after matching to try and reduce this effect

In [171]:
# Merge the matches with the scraped data and the original companies database
result = df_scraped_OK.merge(df_matched_maxScore, left_index=True, right_on='scraped_index')
result = result.merge(df_companies, left_on='company_index', right_index=True, suffixes=('_scraped', '_company'))


In [174]:
result[['url', 'unique_id']]

,url,unique_id
0,https://beeldenbouwers.nl,516268062
1,https://gjgmedia.nl,601569922
2,https://henkvdzanden.nl,524178421
6,https://podotherapierotterdam.nl,605740923
7,https://www.issos.nl/,497161776
...,...,...
6998,https://www.out-smart.eu/,600241291
6999,https://eosmaritime.com,603511814
7001,https://www.a3architecten.com/,603823102
7003,https://kadoburo.nl,514393675


In [175]:
result.to_csv('Result.csv')

# Conclusion

To Do: 
- Improve the fuzzy matching to remove good matches and reduce number of repetitive matches
- Improve on matching using phone and copyright info
- Improve on scraping for phone info using regex or other improvements